In [ ]:
# a very simple example of factory pattern
class EmailNotification:
    def send(self):
        print("Sending Email")

class SMSNotification:
    def send(self):
        print("Sending SMS")

notif = EmailNotification()
notif.send()

# Problem:

# Your code must know exact class names

# If tomorrow you add PushNotification, you must modify all places

In [ ]:
# Solution: Factory Pattern
# Step 1 : Define a common interface (base class)

class Notification:    
    """
    the following is the use of this base class:
    Any object created by the factory must have a send() method
    lets assume a new WhatsappNotification class is created tomorrow, then someone might create deliver() method instead of send()
    From a library POV, we want to make sure that any class created by the factory has a send() method
    NOTE: one can use deliver() in new sub-classes, but if someone uses send then error is thrown

    ALSO, if someone wants to do the following:
    def notify(notification: Notification):
        notification.send()
    This function:
        Doesnt care whether its Email, SMS, Push
        Only cares that its a Notification
    
    """
    def send(self):
        raise NotImplementedError
    
# Step 2 : Create concrete classes (actual classes) implementing the same interface
class EmailNotification(Notification):
    def send(self):
        print("Sending Email")

class SMSNotification(Notification):
    def send(self):
        print("Sending SMS")

# Step 3: Create the Factory
class NotificationFactory:
    @staticmethod
    def create_notification(notification_type):
        if notification_type == "email":
            return EmailNotification()
        elif notification_type == "sms":
            return SMSNotification()
        else:
            raise ValueError("Unknown notification type")

# Step 4: Use the Factory to get object of concrete class by passing an information such as type
notif = NotificationFactory.create_notification("email")
notif.send()  # Output: Sending Email

# If tomorrow you add PushNotification, you just need to add a new class and modify the factory only.
notif2 = NotificationFactory.create_notification("sms")
notif2.send()  # Output: Sending SMS    


Sending Email
Sending SMS


In [ ]:
class NotificationFactory:
    _creators = {
        "email": EmailNotification,
        "sms": SMSNotification
    }

    @staticmethod
    def create_notification(notification_type):
        creator = NotificationFactory._creators.get(notification_type)
        if not creator:
            raise ValueError("Unknown notification type")
        return creator()

# Slightly Better Pythonic Factory (Using Dictionary) instead of simple if/else



# GENERAL FACTORY PATTERN TEMPLATE
BASE CLASS (BETTER BE AN ABSTRACT CLASS)

ORIGINAL YOUR CLASSES (BUT SUB-CLASSES)

FACTORY CLASS



YOUR/clients CODE WHERE OBJECTS ARE CREATED


In [ ]:
from abc import ABC, abstractmethod

class Notification(ABC):
    @abstractmethod
    def send(self):
        pass

class EmailNotification(Notification):
    pass

EmailNotification()  # ❌ TypeError

# with abstract method, we can ensure there has to be a send() method


TypeError: Can't instantiate abstract class EmailNotification without an implementation for abstract method 'send'

## Example 1: Simple Factory Pattern

source : https://realpython.com/factory-method-python/

Factory Method separates the process of creating an object from the code that depends on the interface of the object.
Instead of using a complex if/elif/else conditional structure to determine the concrete implementation (the original implementation of a class), the application delegates that decision to a separate component that creates the concrete object. With this approach, the application code is simplified, making it more reusable and easier to maintain.

Generally used where we have similar set of classes and its objects (such as bus, truck, car belonging to Vehicle category) to be created.

In [ ]:
# the following is a Song class and see how it gets refined.
# the goal of this code is to serialize song information into different formats (JSON and XML).
# instead of object creation, here we are dealing with optimizing if/else blocks for different formats.

import json
import xml.etree.ElementTree as et

class Song:
    def __init__(self, song_id, title, artist):
        self.song_id = song_id
        self.title = title
        self.artist = artist


class SongSerializer:
    def serialize(self, song, format):
        if format == 'JSON':
            song_info = {
                'id': song.song_id,
                'title': song.title,
                'artist': song.artist
            }
            return json.dumps(song_info)
        elif format == 'XML':
            song_info = et.Element('song', attrib={'id': song.song_id})
            title = et.SubElement(song_info, 'title')
            title.text = song.title
            artist = et.SubElement(song_info, 'artist')
            artist.text = song.artist
            return et.tostring(song_info, encoding='unicode')
        else:
            raise ValueError(format)

Complex logical code uses if/elif/else structures to change the behavior of an application. Using if/elif/else conditional structures makes the code harder to read, harder to understand, and harder to maintain.

Let’s take a look at all the situations that will require modifications to the implementation:

    When a new format is introduced: The method will have to change to implement the serialization to that format.

    When the Song object changes: Adding or removing properties to the Song class will require the implementation to change in order to accommodate the new structure.

    When the string representation for a format changes (plain JSON vs JSON API): The .serialize() method will have to change if the desired string representation for a format changes because the representation is hard-coded in the .serialize() method implementation.


In [ ]:
class SongSerializer:
    def serialize(self, song, format):
        if format == 'JSON':
            return self._serialize_to_json(song)    # THE BLOCK IS MOVED TO A SEPARATE METHOD
        elif format == 'XML':
            return self._serialize_to_xml(song)     # THE BLOCK IS MOVED TO A SEPARATE METHOD
        else:
            raise ValueError(format)

    def _serialize_to_json(self, song):
        payload = {
            'id': song.song_id,
            'title': song.title,
            'artist': song.artist
        }
        return json.dumps(payload)

    def _serialize_to_xml(self, song):
        song_element = et.Element('song', attrib={'id': song.song_id})
        title = et.SubElement(song_element, 'title')
        title.text = song.title
        artist = et.SubElement(song_element, 'artist')
        artist.text = song.artist
        return et.tostring(song_element, encoding='unicode')

The central idea in Factory Method is to provide a separate component with the responsibility to decide which concrete implementation should be used based on some specified parameter. 

In [ ]:
class SongSerializer:
    def serialize(self, song, format):
        serializer = self._get_serializer(format)  # THE BLOCK IS AGAIN MOVED TO A SEPARATE METHOD
        return serializer(song)

    def _get_serializer(self, format):             # THE ACTUAL CREATOR COMPONENT
        if format == 'JSON':                       # THE IF/ELSE STILL EXISTS, BUT MOVED TO A SEPARATE METHOD
            return self._serialize_to_json
        elif format == 'XML':
            return self._serialize_to_xml
        else:
            raise ValueError(format)

    def _serialize_to_json(self, song):
        payload = {
            'id': song.song_id,
            'title': song.title,
            'artist': song.artist
        }
        return json.dumps(payload)

    def _serialize_to_xml(self, song):
        song_element = et.Element('song', attrib={'id': song.song_id})
        title = et.SubElement(song_element, 'title')
        title.text = song.title
        artist = et.SubElement(song_element, 'artist')
        artist.text = song.artist
        return et.tostring(song_element, encoding='unicode')

Factory pattern need not be used only during the object creation, but also in the following cases:
1. Replacing complex logical code: Complex logical structures in the format if/elif/else are hard to maintain because new logical paths are needed as requirements change. Within all the if/else, there is a similar type of tasks performed.
2. Integrating related external services: A music player application wants to integrate with multiple external services and allow users to select where their music comes from. The application can define a common interface for a music service and use Factory Method to create the correct integration based on a user preference.
3. Supporting multiple implementations of the same feature: An image processing application needs to transform a satellite image from one coordinate system to another, but there are multiple algorithms with different levels of accuracy to perform the transformation.

The application can allow the user to select an option that identifies the concrete algorithm. Factory Method can provide the concrete implementation of the algorithm based on this option.

4. Combining similar features under a common interface: Following the image processing example, an application needs to apply a filter to an image. The specific filter to use can be identified by some user input, and Factory Method can provide the concrete filter implementation.

In all the above examples, he is referring to that one specific function that can manage if/else etc.

for more detailed, see the Real python site on the following implementation


In [ ]:
# here factory class is kept very generic such that anyone can register their own classes and then this can be used to create objects of those classes

class ObjectFactory:
    def __init__(self):
        self._builders = {}

    def register_builder(self, key, builder):
        self._builders[key] = builder

    def create(self, key, **kwargs):
        builder = self._builders.get(key)
        if not builder:
            raise ValueError(key)
        return builder(**kwargs)
    




# Omitting other implementation classes shown below

factory = ObjectFactory()
factory.register_builder('SPOTIFY', SpotifyServiceBuilder())  # one can register which is nothing but mapping key to class or function
factory.register_builder('PANDORA', PandoraServiceBuilder())
factory.register_builder('LOCAL', create_local_music_service)

# sample example of creating object
spotify2 = music.services.get('SPOTIFY', **config) # config is just some dictionary of parameters